# **Importing**

In [24]:
!pip install nltk

In [2]:
import pandas as pd
import numpy as np
import ast
from nltk.stem import WordNetLemmatizer
# from nltk.corpus import stopwords
import spacy
import nltk
nltk.download('wordnet')
# import the TfIdfVectorizer from scikit-learn library
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [51]:
!unzip /content/movies.csv.zip

Archive:  /content/movies.csv.zip
replace movies.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [3]:
df =  pd.read_csv('/content/movies.csv')

In [5]:
df.head()

,movie_id,description,language,released,rating,writer,director,cast,genre,name
0,tt5286444,Neerja is the story of the courageous Neerja B...,"[ ""Hindi"" ]",2016-02-19T00:00:00.000Z,7.9,"[ ""Saiwyn Quadras"", ""Sanyukta Shaikh Chawla"" ]","[ ""Ram Madhvani"" ]","[ ""Sonam Kapoor"", ""Shabana Azmi"", ""Yogendra Ti...","[ ""Biography"", ""Drama"", ""Thriller"" ]",Neerja
1,tt4434004,A story that revolves around drug abuse in the...,"[ ""Hindi"", ""Panjabi"" ]",2016-06-17T00:00:00.000Z,7.9,"[ ""Sudip Sharma"", ""Abhishek Chaubey"" ]","[ ""Abhishek Chaubey"" ]","[ ""Alia Bhatt"", ""Shahid Kapoor"", ""Diljit Dosan...","[ ""Crime"", ""Drama"", ""Thriller"" ]",Udta Punjab
2,tt0248126,Yashvardhan Raichand lives a very wealthy life...,"[ ""Hindi"", ""English"", ""Urdu"" ]",2001-12-14T00:00:00.000Z,7.5,"[ ""Karan Johar"", ""Sheena Parikh"" ]","[ ""Karan Johar"" ]","[ ""Amitabh Bachchan"", ""Jaya Bhaduri"", ""Shah Ru...","[ ""Drama"", ""Musical"", ""Romance"" ]",Kabhi Khushi Kabhie Gham...
3,tt0347304,"Naina, an introverted, perpetually depressed g...","[ ""Hindi"", ""Urdu"", ""Gujarati"", ""Panjabi"" ]",2003-11-28T00:00:00.000Z,8.0,"[ ""Niranjan Iyengar"", ""Karan Johar"" ]","[ ""Nikkhil Advani"" ]","[ ""Shah Rukh Khan"", ""Preity Zinta"", ""Saif Ali ...","[ ""Comedy"", ""Drama"", ""Romance"" ]",Kal Ho Naa Ho
4,tt3043252,'Parched' is a story about women set in the he...,"[ ""Hindi"", ""English"" ]",2016-06-17T00:00:00.000Z,7.6,"[ ""Supratik Sen"", ""Leena Yadav"" ]","[ ""Leena Yadav"" ]","[ ""Tannishtha Chatterjee"", ""Radhika Apte"", ""Le...","[ ""Drama"" ]",Parched


# **Preprocessing**

In [6]:
df.shape

(2850, 10)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2850 entries, 0 to 2849
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   movie_id     2850 non-null   object 
 1   description  1752 non-null   object 
 2   language     2850 non-null   object 
 3   released     2199 non-null   object 
 4   rating       2850 non-null   float64
 5   writer       2850 non-null   object 
 6   director     2850 non-null   object 
 7   cast         2850 non-null   object 
 8   genre        2850 non-null   object 
 9   name         2850 non-null   object 
dtypes: float64(1), object(9)
memory usage: 222.8+ KB


In [8]:
df.isnull().sum()

,0
movie_id,0
description,1098
language,0
released,651
rating,0
writer,0
director,0
cast,0
genre,0
name,0


In [9]:
# dropping null rows
df.dropna(inplace=True)

In [10]:
df.shape

(1586, 10)

In [11]:
df.duplicated(subset=['movie_id']).sum()
# no duplicates based on movie id

0

In [12]:
# dropping rating, writer and released column
new_df = df.drop(['rating','released','writer'],axis=1)

In [13]:
new_df.head()

,movie_id,description,language,director,cast,genre,name
0,tt5286444,Neerja is the story of the courageous Neerja B...,"[ ""Hindi"" ]","[ ""Ram Madhvani"" ]","[ ""Sonam Kapoor"", ""Shabana Azmi"", ""Yogendra Ti...","[ ""Biography"", ""Drama"", ""Thriller"" ]",Neerja
1,tt4434004,A story that revolves around drug abuse in the...,"[ ""Hindi"", ""Panjabi"" ]","[ ""Abhishek Chaubey"" ]","[ ""Alia Bhatt"", ""Shahid Kapoor"", ""Diljit Dosan...","[ ""Crime"", ""Drama"", ""Thriller"" ]",Udta Punjab
2,tt0248126,Yashvardhan Raichand lives a very wealthy life...,"[ ""Hindi"", ""English"", ""Urdu"" ]","[ ""Karan Johar"" ]","[ ""Amitabh Bachchan"", ""Jaya Bhaduri"", ""Shah Ru...","[ ""Drama"", ""Musical"", ""Romance"" ]",Kabhi Khushi Kabhie Gham...
3,tt0347304,"Naina, an introverted, perpetually depressed g...","[ ""Hindi"", ""Urdu"", ""Gujarati"", ""Panjabi"" ]","[ ""Nikkhil Advani"" ]","[ ""Shah Rukh Khan"", ""Preity Zinta"", ""Saif Ali ...","[ ""Comedy"", ""Drama"", ""Romance"" ]",Kal Ho Naa Ho
4,tt3043252,'Parched' is a story about women set in the he...,"[ ""Hindi"", ""English"" ]","[ ""Leena Yadav"" ]","[ ""Tannishtha Chatterjee"", ""Radhika Apte"", ""Le...","[ ""Drama"" ]",Parched


In [14]:
df['cast'].apply(lambda x: len(x.split(','))).unique()

array([15, 12,  8, 10, 11,  9, 14, 13,  7,  5,  4,  6,  3,  1,  2])

In [15]:
# converting lists stored in str format into list using ast.literal_eval
def convert_str_to_list(x):
  return ast.literal_eval(x)


In [16]:
new_df['language'] = df['language'].apply( convert_str_to_list)
new_df['genre'] = df['genre'].apply( convert_str_to_list)
new_df['cast'] = df['cast'].apply( convert_str_to_list)
new_df['director'] = df['director'].apply( convert_str_to_list)

In [17]:
# keep only the first 4 cast (actors) in the list format
# keeping only the 1st director

new_df['cast'] = new_df['cast'].apply(lambda x: x[:4] if len(x) > 4 else x)
new_df['director'] = new_df['director'].apply(lambda x: x[:1])
new_df.head()

,movie_id,description,language,director,cast,genre,name
0,tt5286444,Neerja is the story of the courageous Neerja B...,[Hindi],[Ram Madhvani],"[Sonam Kapoor, Shabana Azmi, Yogendra Tikku, A...","[Biography, Drama, Thriller]",Neerja
1,tt4434004,A story that revolves around drug abuse in the...,"[Hindi, Panjabi]",[Abhishek Chaubey],"[Alia Bhatt, Shahid Kapoor, Diljit Dosanjh, Ka...","[Crime, Drama, Thriller]",Udta Punjab
2,tt0248126,Yashvardhan Raichand lives a very wealthy life...,"[Hindi, English, Urdu]",[Karan Johar],"[Amitabh Bachchan, Jaya Bhaduri, Shah Rukh Kha...","[Drama, Musical, Romance]",Kabhi Khushi Kabhie Gham...
3,tt0347304,"Naina, an introverted, perpetually depressed g...","[Hindi, Urdu, Gujarati, Panjabi]",[Nikkhil Advani],"[Shah Rukh Khan, Preity Zinta, Saif Ali Khan, ...","[Comedy, Drama, Romance]",Kal Ho Naa Ho
4,tt3043252,'Parched' is a story about women set in the he...,"[Hindi, English]",[Leena Yadav],"[Tannishtha Chatterjee, Radhika Apte, Lehar Kh...",[Drama],Parched


In [18]:
# creating one word for name and surname to keep uniqueness intact

new_df['cast'] = new_df['cast'].apply(lambda x: [i.replace(' ','') for i in x])
new_df['director'] = new_df['director'].apply(lambda x: [i.replace(' ','') for i in x])

In [19]:
new_df.head()

,movie_id,description,language,director,cast,genre,name
0,tt5286444,Neerja is the story of the courageous Neerja B...,[Hindi],[RamMadhvani],"[SonamKapoor, ShabanaAzmi, YogendraTikku, Abra...","[Biography, Drama, Thriller]",Neerja
1,tt4434004,A story that revolves around drug abuse in the...,"[Hindi, Panjabi]",[AbhishekChaubey],"[AliaBhatt, ShahidKapoor, DiljitDosanjh, Karee...","[Crime, Drama, Thriller]",Udta Punjab
2,tt0248126,Yashvardhan Raichand lives a very wealthy life...,"[Hindi, English, Urdu]",[KaranJohar],"[AmitabhBachchan, JayaBhaduri, ShahRukhKhan, K...","[Drama, Musical, Romance]",Kabhi Khushi Kabhie Gham...
3,tt0347304,"Naina, an introverted, perpetually depressed g...","[Hindi, Urdu, Gujarati, Panjabi]",[NikkhilAdvani],"[ShahRukhKhan, PreityZinta, SaifAliKhan, JayaB...","[Comedy, Drama, Romance]",Kal Ho Naa Ho
4,tt3043252,'Parched' is a story about women set in the he...,"[Hindi, English]",[LeenaYadav],"[TannishthaChatterjee, RadhikaApte, LeharKhan,...",[Drama],Parched


In [20]:
# joining all the elements in language, director, cast, genre columns stored in lists with spaces and description column which is already in str format
def create_corpus(x):
  return " ".join(x['language']) + " " + " ".join(x['director']) + " " + " ".join(x['cast']) + " " + " ".join(x['genre']) + " " + x['description']

In [21]:
new_df['corpus'] = new_df.apply(create_corpus, axis=1)

In [22]:
# converting all the words in the corpus into lowecase
new_df['corpus'] = new_df['corpus'].apply( lambda x : x.lower())

In [23]:
# dropping all the columns used to make the corpus
new_df.drop(['language','genre','director','description','cast'],axis=1,inplace=True)

In [24]:
new_df.head()

,movie_id,name,corpus
0,tt5286444,Neerja,hindi rammadhvani sonamkapoor shabanaazmi yoge...
1,tt4434004,Udta Punjab,hindi panjabi abhishekchaubey aliabhatt shahid...
2,tt0248126,Kabhi Khushi Kabhie Gham...,hindi english urdu karanjohar amitabhbachchan ...
3,tt0347304,Kal Ho Naa Ho,hindi urdu gujarati panjabi nikkhiladvani shah...
4,tt3043252,Parched,hindi english leenayadav tannishthachatterjee ...


In [28]:
# lemmatization
def lemmatize_words(text):

    lemmatizer = WordNetLemmatizer()

    # Load the spaCy English model
    nlp = spacy.load('en_core_web_sm')

    # Process the text with spaCy
    doc =  nlp(text)


    lemmatized_words = []
    for word in doc:
      lemmatized_words.append(word.lemma_)

    return ' '.join(lemmatized_words)

In [27]:
new_df['corpus'][0]

'hindi rammadhvani sonamkapoor shabanaazmi yogendratikku abrarzahoor biography drama thriller neerja is the story of the courageous neerja bhanot, who sacrificed her life while protecting the lives of 359 passengers on the pan am flight 73 in 1986. the flight was hijacked by a terrorist organization.'

In [29]:
print(lemmatize_words(new_df['corpus'][0]))

hindi rammadhvani sonamkapoor shabanaazmi yogendratikku abrarzahoor biography drama thriller neerja be the story of the courageous neerja bhanot , who sacrifice her life while protect the life of 359 passenger on the pan am flight 73 in 1986 . the flight be hijack by a terrorist organization .


In [30]:
new_df['corpus'] = new_df['corpus'].apply(lemmatize_words)

In [31]:
new_df.head()

,movie_id,name,corpus
0,tt5286444,Neerja,hindi rammadhvani sonamkapoor shabanaazmi yoge...
1,tt4434004,Udta Punjab,hindi panjabi abhishekchaubey aliabhatt shahid...
2,tt0248126,Kabhi Khushi Kabhie Gham...,hindi english urdu karanjohar amitabhbachchan ...
3,tt0347304,Kal Ho Naa Ho,hindi urdu gujarati panjabi nikkhiladvani shah...
4,tt3043252,Parched,hindi english leenayadav tannishthachatterjee ...


In [35]:
new_df['corpus'][22]

'hindi english urdu farhanakhtar aamirkhan saifalikhan akshayekhanna preityzinta comedy drama romance three inseparable childhood friend be just out of college . nothing come between they - until they each fall in love , and their wildly different approach to relationship create tension .'

# **TF IDF Vectorizer**

In [50]:
# define a TF IDF Vectorizer Object
# with the removal of english stopwords turned on
tfidf = TfidfVectorizer(stop_words = 'english')

# creating the TF-IDF matrix
tfidf_matrix = tfidf.fit_transform(new_df['corpus']).toarray()

tfidf_matrix.shape
# 1586 movies and 10210 words

(1586, 10210)

In [51]:
tfidf_matrix
# creating a vector of size 10210 decribing the content of each movie

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

# **Calculating Similarity scores for content based recommendation**

In [54]:
# creating similarity matrix
similarity = cosine_similarity(tfidf_matrix)

In [55]:
similarity

array([[1.        , 0.02109316, 0.00510146, ..., 0.02345491, 0.01665403,
        0.00156146],
       [0.02109316, 1.        , 0.00625134, ..., 0.02148583, 0.00199966,
        0.00191342],
       [0.00510146, 0.00625134, 1.        , ..., 0.00840004, 0.02928289,
        0.00179303],
       ...,
       [0.02345491, 0.02148583, 0.00840004, ..., 1.        , 0.02642131,
        0.06102786],
       [0.01665403, 0.00199966, 0.02928289, ..., 0.02642131, 1.        ,
        0.1098886 ],
       [0.00156146, 0.00191342, 0.00179303, ..., 0.06102786, 0.1098886 ,
        1.        ]])

In [56]:
similarity.shape
# similarity of each movie with every other movie
# cosine distance of the vector of each movie with every other movie

(1586, 1586)

In [61]:
new_df

,movie_id,name,corpus
0,tt5286444,Neerja,hindi rammadhvani sonamkapoor shabanaazmi yoge...
1,tt4434004,Udta Punjab,hindi panjabi abhishekchaubey aliabhatt shahid...
2,tt0248126,Kabhi Khushi Kabhie Gham...,hindi english urdu karanjohar amitabhbachchan ...
3,tt0347304,Kal Ho Naa Ho,hindi urdu gujarati panjabi nikkhiladvani shah...
4,tt3043252,Parched,hindi english leenayadav tannishthachatterjee ...
...,...,...,...
2838,tt5872120,Run Antony,kannada raghushastry vinayrajkumar ruksarmeer ...
2841,tt3666724,Jolly Days,kannada m.d.sridhar pradeepbogadi aishwaryanag...
2842,tt5652478,The Great Story of Sodabuddi,kannada jyothiraomohith uthpal anusha khushi r...
2844,tt5508936,Vamshi,kannada prakash puneethrajkumar nikitathukral ...


# **Main function**

In [58]:
def recommend(movie):
  # getting the index of the movie drom the dataframe
  movie_index = new_df[new_df['name'] == movie].index[0]

  # fetching similarity score from the similarity matrix using the index
  # coverting it into enumerate object to fetch the index of movie along with the similarity score
  # sorting the similarity scores in descending order
  distances = sorted(list(enumerate(similarity[movie_index])), reverse=True, key = lambda x: x[1])

  for i in distances[1:6]:
    # fetching the names of recommended movies
        print(new_df.iloc[i[0],1])

In [59]:
recommend('Dil Se..')

Veer-Zaara
Bombay
Dil Chahta Hai
Kal Ho Naa Ho
Darr


# **Creating pkl files**

In [60]:
# dumping new_df and similarity matrix into pickle files
pickle.dump(new_df, open('movie_list.pkl','wb'))
pickle.dump(similarity, open('similarity.pkl','wb'))